In [ ]:
# fix model checkpoint at each epoch - FIXED
# fix tensorboard - FIXED
# make in advance a model and py script for each training set
# testing and plotting to understand results

# test how much r2 are we achieving with only sequences - so only LSTM to output, 
# although still convolution to accelerate learning

# approx 180 epochs training - > training over 100 does increase performance
# have to implement R2 as performance metric

In [1]:
import numpy as np
from numpy import argmax
import pandas as pd
#import h5py
from time import time
from keras.models import Model, load_model, Sequential
from keras.layers import Conv1D,MaxPooling1D,LSTM,BatchNormalization,Dropout,Input,Dense,Bidirectional,Activation,Add,Concatenate,Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.backend import squeeze
from keras.utils import io_utils, HDF5Matrix
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger

Using TensorFlow backend.


In [2]:
#checks if running notebook or standalone python script
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

import configparser
import sys
args = (sys.argv)

model_name = "THIS_model"
DROPOUT = 0   # dropout
ALPHA = 0.01 # learnrate
BETA = 0.01
EPOCHS = 2 # epochs
MBATCH = 100 # batch size

#loading from config 
if not is_interactive():
    config_file = args[1]
    config = configparser.ConfigParser()
    config.read(config_file)
    ALPHA = config.getfloat('main', 'alpha')
    DROPOUT = config.getfloat('main', 'dropout')
    EPOCHS = config.getint('main', 'epochs')
    MBATCH = config.getint('main', 'mbatch')
    BETA = config.getfloat('main', 'beta')


In [3]:
## load data
data_path = "../data/THIS_data.npz"

if not is_interactive():
    data_path = "./data/THIS_data.npz"
npzfile = np.load(data_path)
Xh_train = npzfile['arr_0']
Xh_test = npzfile['arr_1']
Xv_train = npzfile['arr_2']
Xv_test = npzfile['arr_3']
Y_train = npzfile['arr_4']
Y_test = npzfile['arr_5']

print(Xh_train.shape)
print(Xv_train.shape)

(4006, 650, 4)
(4006, 69)


In [4]:
#shape1 = Xh_train.shape
#shape2 = Xv_train.shape

In [5]:
# x: Numpy array of training data (if the model has a single input), or list of Numpy arrays 
# (if the model has multiple inputs). If input layers in the model are named, you can also 
# pass a dictionary mapping input names to Numpy arrays. 
X_train = list()
X_train.append(Xh_train)
#X_train.append(Xv_train)

X_test = list()
X_test.append(Xh_test)
#X_test.append(Xv_test)

In [6]:
def POC_model_2(input_shape_hot):
    # model has 2 sets of separate input layers
    
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input1 = Input(shape = input_shape_hot)
    #X_input2 = Input(shape = input_shape_val)
    DR = DROPOUT
    
    ## Step 1 - hot layers
    # L 1: CONV 
    X1 = Conv1D(filters=16, kernel_size=10, strides=3)(X_input1) # size-kernel/stride + 1 = 640/2 + 1 = 321 out
    X1 = BatchNormalization()(X1)
    X1 = Dropout(DR)(X1)   # dropout
    
    # L 2: Maxpool
    X1 = MaxPooling1D(pool_size=16, strides=2)(X1) # 321/2 +1 = 162 (x 16) out                 
    X1 = BatchNormalization()(X1)
    #X = Activation('relu')(X)  # ReLu activation, based on other models from course
    X1 = Dropout(DR)(X1)
    
    # L 3: Bidirectional LSTM
    X1 = Bidirectional(LSTM(81, return_sequences=True))(X1) # 81 out
    X1 = BatchNormalization()(X1)
    #X = Activation('relu')(X)
    X1 = Dropout(DR)(X1) 
    
    # L 3: Bidirectional LSTM
    X1 = Bidirectional(LSTM(40, return_sequences=False))(X1) # 40 out
    X1 = BatchNormalization()(X1)
    #X = Activation('relu')(X)
    X1 = Dropout(DR)(X1)
    
    # Step 2 - neural network merge data
    # L1 -2 Dense layer for 
    #X = Concatenate(axis=1)([X1,X_input2])
    X = Dense(20)(X1)  # 20 out
    X = BatchNormalization()(X)
    X = Dropout(DR)(X) 

    # Step 3 - output
    X = Dense(1)(X)
    #X = Activation('softmax')(X) # activation depending on classification problem solved

    model = Model(inputs = [X_input1], outputs = X)
    
    return model

In [7]:
model = POC_model_2((650,4))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 650, 4)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 214, 16)           656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 214, 16)           64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 214, 16)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 16)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 100, 16)           64        
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 16)           0         
__________

In [8]:
from keras import backend as K
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true-y_pred))
    SS_tot = K.sum(K.square(y_true-K.mean(y_true)))
    return (1-SS_res/(SS_tot+K.epsilon()))

import tensorflow as tf

def correlation_coefficient(y_true, y_pred):
    pearson_r, update_op = tf.contrib.metrics.streaming_pearson_correlation(y_pred, y_true, name='pearson_r')
    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'pearson_r'  in i.name.split('/')]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        pearson_r = tf.identity(pearson_r)
        return pearson_r*abs(pearson_r)



In [9]:
opt = Adam(lr=ALPHA, beta_1=0.9, beta_2=0.999, decay=BETA)
model.compile(loss='mse', optimizer=opt, metrics=[coeff_determination, correlation_coefficient, 'mse'])

In [10]:
# Set callbacks
# checkpoint
# https://machinelearningmastery.com/check-point-deep-learning-models-keras/
# https://keras.io/callbacks/ - for now save every epoch
suffix = "{epoch:03d}-{val_loss:.3f}.hdf5"
filepath = "{}.{}".format(model_name, suffix)
if not is_interactive():
    model_name = args[2]

    filepath = "{}.{}".format(model_name, suffix)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, 
                             save_best_only=False, save_weights_only=False, mode='auto', period=1)

# tensorboard
# http://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/

tensorboard = TensorBoard(log_dir='./tensorboard_logs/run1')

csv_logger_filename = "csvresults_log.csv"
if not is_interactive():
    csv_logger_filename = args[3]
csv = CSVLogger(csv_logger_filename, separator = ",", append = True)

callbacks_list = [checkpoint, tensorboard, csv]

# in terminal run: tensorboard --logdir=logs/
# val_loss error is in callbacks, probably modelcheckpoint

In [12]:
#checking if model exist then load best
import glob
import re

def find_best_model(all_models):
        epochs = []
        losses = []
        for i, file in enumerate(all_models):
            groups = re.findall(model_name+'.(.*)-(.*).hdf5', file)
            if groups:
                epochs.append(int(groups[0][0]))
                losses.append(float(groups[0][1]))
        return (all_models[np.argmin(losses)] )

all_models = glob.glob(model_name+'*.hdf5')

if all_models:
    best_model = find_best_model(all_models)
    print("Loading weights from {}".format(best_model))
    model.load_weights(best_model)


model.fit(X_train, Y_train, batch_size=MBATCH, epochs=EPOCHS, validation_split=0.1, shuffle=False, callbacks=callbacks_list)
# keras model checkpoint KeyError: 'val_loss'
# fix: https://github.com/keras-team/keras/issues/6104
# must add validation_split=xx

Train on 3605 samples, validate on 401 samples
Epoch 1/2
3605/3605 [==============================] - 32s 9ms/step - loss: 23.6841 - coeff_determination: -29.0525 - correlation_coefficient: 4.1726e-04 - mean_squared_error: 23.6841 - val_loss: 6.8327 - val_coeff_determination: -253877.2519 - val_correlation_coefficient: 4.4308e-06 - val_mean_squared_error: 6.8327
Epoch 2/2
3605/3605 [==============================] - 32s 9ms/step - loss: 2.5437 - coeff_determination: -2.2170 - correlation_coefficient: 9.7090e-05 - mean_squared_error: 2.5437 - val_loss: 4.4692 - val_coeff_determination: -75796.6565 - val_correlation_coefficient: -8.5414e-07 - val_mean_squared_error: 4.4692


In [13]:
loss,r2, pearson, mse,  = model.evaluate(X_test, Y_test)
print("Dev set mse = ", mse)
print("Dev set pearson = ", pearson)
print("Dev set r2 = ", r2)

446/446 [==============================] - 2s 4ms/step
Dev set mse =  4.29579925109
Dev set pearson =  -2.31320497851e-05
Dev set r2 =  -5.87566538777


In [14]:
Y_pred = model.predict(X_test)
#print(Y_pred)

In [15]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
# https://stackoverflow.com/questions/893657/how-do-i-calculate-r-squared-using-python-and-numpy
from scipy import stats
slope, intercept, rvalue, pvalue, stderr = stats.linregress(Y_test[:,0], Y_pred[:,0])

In [16]:
print("r-squared:", rvalue*abs(rvalue))

r-squared: -0.0024365668515
